# Make dataset

In [1]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from sec_edgar_functions import (
    get_shares_repurchase, 
    get_cashflow_sbc, 
    get_diluted_shares_outstanding,
)

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', None)

In [ ]:
tickers = [
    # "UBER",
    # "LYFT"
    # "SNOW",
    # "CRM",
    # "PINS",
    "SNAP"
]

In [3]:
def get_data(ticker: str) -> pd.DataFrame:
    # Get from API
    shares_repurchase = get_shares_repurchase(ticker, "quarterly")
    stock_based_compensation = get_cashflow_sbc(ticker, "quarterly")
    diluted_shares_outstanding = get_diluted_shares_outstanding(ticker, "quarterly")

    # Parse data
    shares_repurchase = shares_repurchase.T.reset_index()
    shares_repurchase.columns = ['date'] + list(shares_repurchase.columns[1:])

    stock_based_compensation = stock_based_compensation.T.reset_index()
    stock_based_compensation.columns = ['date'] + list(stock_based_compensation.columns[1:])

    diluted_shares_outstanding = diluted_shares_outstanding.T.reset_index()
    diluted_shares_outstanding.columns = ['date'] + list(diluted_shares_outstanding.columns[1:])

    # Parse data
    shares_repurchase['date'] = pd.to_datetime(shares_repurchase['date'])
    stock_based_compensation['date'] = pd.to_datetime(stock_based_compensation['date'])
    diluted_shares_outstanding['date'] = pd.to_datetime(diluted_shares_outstanding['date'])

    df = pd.merge(shares_repurchase, stock_based_compensation, how='right', on='date')

    # Merge the data
    shares_col = [col for col in diluted_shares_outstanding.columns if col != 'date'][0]

    df = df.sort_values('date').reset_index(drop=True)
    diluted_shares_outstanding_sorted = diluted_shares_outstanding.sort_values('date').reset_index(drop=True)

    shares_df = diluted_shares_outstanding_sorted[['date', shares_col]].copy()
    shares_df = shares_df.rename(columns={shares_col: 'shares_outstanding'})

    if 'shares_outstanding' in df.columns:
        df = df.drop(columns=['shares_outstanding'])

    # Try backward merge first (preferred - uses most recent available data)
    df = pd.merge_asof(
        df,
        shares_df,
        on='date',
        direction='backward' 
    )
    
    # If backward merge left NaNs (e.g., when we only have future data points),
    # use the most recent available value from shares_df to fill backwards
    # This handles cases like LYFT where we only have 2025-10-31 data point
    if df['shares_outstanding'].isna().any():
        # Get the most recent (last) value from shares_df
        if not shares_df.empty and not shares_df['shares_outstanding'].isna().all():
            # Sort by date descending to get the most recent value
            shares_df_sorted = shares_df.sort_values('date', ascending=False)
            most_recent_value = shares_df_sorted['shares_outstanding'].iloc[0]
            if pd.notna(most_recent_value):
                # Fill all NaNs with the most recent known value
                df['shares_outstanding'] = df['shares_outstanding'].fillna(most_recent_value)

    # Fill NaNs with 0 for all columns except shares_outstanding
    shares_outstanding_series = df['shares_outstanding'].copy()
    df = df.fillna(0)
    df['shares_outstanding'] = shares_outstanding_series

    return df

In [4]:
def get_closing_price(ticker: str, date: datetime) -> float:
    end = date + timedelta(days=1)
    start = date - timedelta(days=10)  

    data = yf.download(ticker, start=start, end=end, auto_adjust=True, progress=False)

    if data.empty:
        raise ValueError(f"No data returned for {ticker} in window {start}..{end}")

    eligible = data.loc[:date]

    if eligible.empty:
        raise ValueError(f"No trading data on or before {date.date()} for {ticker}")

    return eligible["Close"].iloc[-1].item()

In [5]:
def df_to_csv(df: pd.DataFrame, filename: str) -> None:
    df.to_csv(filename, index=False, encoding="utf-8")

### Generate CSV for each ticker

In [6]:
for ticker in tickers:
    df = get_data(ticker)

    # Get closing prices column 
    closing_prices = []
    for date in df['date']:
        try:
            price = get_closing_price(ticker, date)
            closing_prices.append(price)
        except ValueError:
            closing_prices.append(float('nan'))

    df['closing_price'] = closing_prices

    df_to_csv(df, f"{ticker}.csv")
    
    print(f"{ticker} dataset done")


1 Failed download:
['SNAP']: YFPricesMissingError('possibly delisted; no price data found  (1d 2016-03-21 00:00:00 -> 2016-04-01 00:00:00) (Yahoo error = "Data doesn\'t exist for startDate = 1458532800, endDate = 1459483200")')

1 Failed download:
['SNAP']: YFPricesMissingError('possibly delisted; no price data found  (1d 2016-06-20 00:00:00 -> 2016-07-01 00:00:00) (Yahoo error = "Data doesn\'t exist for startDate = 1466395200, endDate = 1467345600")')

1 Failed download:
['SNAP']: YFPricesMissingError('possibly delisted; no price data found  (1d 2016-09-20 00:00:00 -> 2016-10-01 00:00:00) (Yahoo error = "Data doesn\'t exist for startDate = 1474344000, endDate = 1475294400")')


SNAP dataset done
